# Scattering calculations

This example demonstrates the basic principles of performing scattering calcualtions with ARTS. In ARTS, species that scattering radiation are referred to as *scattering species*. In essence, a scattering species defines a mapping from one or several fields of *scattering species properties* to *scattering properties* which form the input to the actual scattering calculation.

## Set up a 1D atmosphere

In [1]:
import numpy as np
import pyarts

ws = pyarts.workspace.Workspace()
ws.PlanetSet(option = "Earth")
ws.atm_fieldInit(toa=100e3)
ws.atm_fieldAddGriddedData(
    key=pyarts.arts.String("t"),
    data=pyarts.arts.GriddedField3.fromxml("planets/Earth/afgl/tropical/t.xml")
)
ws.atm_fieldAddGriddedData(
    key=pyarts.arts.String("p"),
    data=pyarts.arts.GriddedField3.fromxml("planets/Earth/afgl/tropical/p.xml")
)

## Add a field of scattering species properties

In order to perform a simulation with ice particles, it is necessary to first define the *scattering species property* that represents the properties of the scattering species in the atmosphere. Below we define the **mass density** of **ice** as the scattering species property we use to describe the distribution of ice hydrometeors in the atmosphere.

In [2]:
ice_mass_density = pyarts.arts.ScatteringSpeciesProperty("ice", pyarts.arts.ParticulateProperty("MassDensity"))

We then define a GriddedField3 representing the ice mass density profile in the atmosphere and add it to  ``atm_field`` of the workspace.

In [11]:
grids = ws.atm_field["t"].data.grids
z = grids[0]
ice_mass_density_profile = np.zeros_like(z)
ice_mass_density_profile[(z > 5e3) * (z < 10e3)] = 1e-3
ice_mass_density_profile = pyarts.arts.GriddedField3(data=ice_mass_density_profile, grids=grids)
ws.atm_field[ice_mass_density] = ice_mass_density_profile

In [12]:
psd_abel12 = pyarts.arts.MGDSingleMoment(ice_mass_density, 0.22, 2.2, 0.0, 1.0, 180.0, 270.0, False)
psd_wang16 = pyarts.arts.MGDSingleMoment(ice_mass_density, "Wang16", 180.0, 270.0, False)
psd_field19 = pyarts.arts.MGDSingleMoment(ice_mass_density, "Field19", 180.0, 270.0, False)

In [13]:
atm_point = ws.atm_field.at([10e3], [0.0], [0.0])
sizes = np.logspace(1e-6, 1e-2, 41)
scat_species_a = 479.9830 
scat_species_b = 3.0

y_abel12 = psd_abel12.evaluate(atm_point[0], sizes, scat_species_a, scat_species_b)
y_wang16 = psd_abel12.evaluate(atm_point[0], sizes, scat_species_a, scat_species_b)
y_field19 = psd_field19.evaluate(atm_point[0], sizes, scat_species_a, scat_species_b)

RuntimeError: The PSD requires water content from '_MassDensity' but it is not part of the AtmPoint.

In [ ]:
plt.plot(sizes, y_abel12, label="Abel 12")
plt.plot(sizes, y_wang16, label="Wang 16")
plt.plot(sizes, y_field19, label="Field 19")
plt.set_xscale("log")
plt.set_yscale("log")
plt.xlabel("Particle size [m]")
plt.ylabel("Particle number density [m$^{-4}$]")